In [2]:
# Test on COCO val data
%%capture
!unzip /content/drive/My\ Drive/val2017.zip

In [4]:
import tensorflow as tf
import tensorflow_addons as tfa 
import numpy as np 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [5]:
from coco import CocoGenerator
from matcher import HungarianMatcher
from loss import DETRLosses
from model import DETR

In [ ]:
BATCH_SIZE = 1
train_generator = CocoGenerator(
    "coco",
    'train2017',     
    batch_size=BATCH_SIZE,
)

In [ ]:
tf.keras.backend.clear_session()
epochs = 100 
model = DETR(num_classes=91,num_queries=100)  
rundetr = DETRLosses(num_classes = model.num_classes-1)
optimizer = tfa.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4)
 

tf.keras.backend.clear_session()
for epoch in range(epochs):
    print("\n########################### Start of epoch %d ############################" % (epoch,))    

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_generator):
        if [len(i["bboxes"]) for i in y_batch_train]==[0]:
          continue    
        print("EPOCH | STEP","------------- {} | {} -------------".format(epoch,step))
        
        with tf.GradientTape() as tape:            
            print(x_batch_train.shape)
            outputs = model(x_batch_train, training=True)  # Outputs for this minibatch
            
            # Compute the loss value for this minibatch.
           
            loss_value = rundetr.combined_loss_fn(outputs,y_batch_train)
             
        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
 
            

    #Save after every epoch
    # model.save_weights('/content/drive/My Drive/kdetr/keras_detr')

      